In [1]:
#### Step 1 ####
import pandas as pd
import numpy as np
import warnings
import time

overall_start = time.time()

## initialise timing
steps = ['1.0   - Initialise import settings', 
         '1.1   - First row check', 
         '1.2   - Set row limit', 
         '1.3   - Concat Chunks (1 file)',
         '1.4   - Remove []',
         '1.5   - Export',
         '1.1-5 - 1 file time (total)',
         'Overall']

times = {step:[] for step in steps}



#### Step 1: import, remove space before inf, NA rows, ] in last column, [ in nAcid column, ####
#### add header descriptors to each file, export as temp file                               ####

#### 1.0   - Initialise import settings ####
start_time = time.time() # opt

## names of files to filter:
files = ['book_dataset_0','book_test_14']
# files = ['book_dataset_0']
# files = ['book_test_14']
# files = ['book_dataset_0_modifiedsubset_7']

## remove .csv first when using google drive path, also uncomment google drive path - tfiles to get filename from path
# files = ['/content/drive/MyDrive/completed sets/book_dataset_17',
#          '/content/drive/MyDrive/completed sets/book_dataset_18',
#          '/content/drive/MyDrive/completed sets/book_dataset_19',
#          '/content/drive/MyDrive/completed sets/book_dataset_27']
# files = files[2:]


## names of 'cleaner' version of files - (NA rows, [ ] removed): temp_filename
tfiles = ['temp_'+ fname for fname in files]

## google drive path - tfiles
# def find_name(paths):
#     return [path[path.find('book'):] for path in paths]
# tfiles = ['temp_'+ file for file in find_name(files)]


first_file_w_descriptor = 'book_dataset_0'

# to check if all the number of columns are the same
ncol_check = []
times['1.0   - Initialise import settings'].append(time.time() - start_time)





for count, file in enumerate(files):
    file_time = time.time() # opt
    
    
    
    
    
    #### 1.1   - First row check ####
    start_time = time.time() # opt
    
    # checking whether first row is descriptor
    first_row = pd.read_csv('{}.csv'.format(file), nrows = 0, header = 0)
    
    try:
        float(first_row.columns[3])
        print('Not first file, using descriptors from {} as header'.format(first_file_w_descriptor))
        header = pd.read_csv('{}.csv'.format(first_file_w_descriptor), nrows = 0, header = 0)
        descriptors = header.columns
        skiprow = None

    except ValueError:
        print ("This is the first file/a file with header descriptors")
        descriptors = first_row.columns
        # skip first row (header) later since already imported headers
        skiprow = 1
    times['1.1   - First row check'].append(time.time() - start_time)
    
    
    
    
    
    #### 1.2   - Chunk ####
    start_time = time.time() # opt    
    
    # number of rows to read at once
    rchunk_s = 100
    # setting row limit (nrows) prevents pandas to read the empty rows (after data) (happens to book-dataset1, doens't happen to book_dataset_0)
    row_lim = pd.read_csv('{}.csv'.format(file), usecols = [0], names = descriptors, skiprows = skiprow).dropna().shape[0]
    # recognising " 'NA'" as NaN
    navalues = "'NA'"
    times['1.2   - Set row limit'].append(time.time() - start_time)

    
    # read a file in row chunks
    chunks = pd.read_csv('{}.csv'.format(file), names = descriptors, skiprows = skiprow, chunksize = rchunk_s, nrows = row_lim, na_values = navalues, skipinitialspace = True)
    
    # Iteratively appending rows to a DataFrame can be more computationally intensive than a single concatenate.
    # A better solution is to append those rows to a list 
    # and then concatenate the list with the original DataFrame all at once.
    #    https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.append.html
    
    
    
    
    
    #### 1.3   - Concat Chunks (1 file) ####
    start_time = time.time() # opt
    
    # removing NA rows upon import, concat all row chunks (of a single file)
    df  = pd.concat(chunks).dropna()
    
    print('Read {} rows in {}'.format(row_lim, file))
    print('NA rows removed')
    print('{} rows left'.format(df.shape[0]))
    
    times['1.3   - Concat Chunks (1 file)'].append(time.time() - start_time)
    
    # store number of columns of file to be checked later
    ncol_check.append(df.shape[1])
    
    
    
    
    
    #### 1.4   - Remove [] ####
    start_time = time.time() # opt
    
    # removing ] in last column
    if 'PubchemFP880' in df.columns:
        if type(df['PubchemFP880'].iloc[-1]) == str and "]" in df['PubchemFP880'].iloc[-1]:
            df['PubchemFP880'] = pd.to_numeric(df['PubchemFP880'].astype(str).str.rstrip(']'))
            print('] removed in last column')
        else:
            print('] not present in last column')
    else:
        print('PubchemFP880 not among columns')

    # removing [ in nAcid column
    if 'nAcid' in df.columns:
        if type(df['nAcid'].iloc[3]) == str and "[" in df['nAcid'].iloc[0]:
            df['nAcid'] = pd.to_numeric(df['nAcid'].str.lstrip('['))
            print('[ removed in nAcid')
        else:
            print('[ not present in nAcid')
    else:
        print('nAcid not among columns')
    times['1.4   - Remove []'].append(time.time() - start_time)
    
    
    
    
    
    #### 1.5   - Export ####
    start_time = time.time() # opt
    
    df.to_csv('{}.csv'.format(tfiles[count]), index=False)
    print('File exported as {}.csv'.format(tfiles[count]))
    print()
    times['1.5   - Export'].append(time.time() - start_time)
    
    times['1.1-5 - 1 file time (total)'].append(time.time() - file_time)
    
    # clear variable to reduce RAM usage
    chunks = None
    df     = None
    
    
    
    
    
# make sure that all files have same number of cols
if ncol_check.count(ncol_check[0]) == len(ncol_check):
    print('All files have same number ({}) of columns'.format(ncol_check[0]))
else:
    print('****Warning: files have different numbers of columns****')
    for tfile, ncol in zip(tfiles, ncol_check):
        print ('{}: {} columns'.format(tfile, ncol))


        
        
## file naming
# take numbers only from each file
nums = []
for tfile in tfiles:
    fnums = [char for char in tfile if char.isdigit()]
    nums.append(''.join(fnums))

if len(nums) < 5:
    nums_ = '_'.join(nums)
else: # 5 files or above
    nums_ = '_'.join(nums[:2] +['...'] + nums[-2:])
    
print(tfiles)
times['Overall'].append(time.time() - overall_start)
print("--- {:.4f} seconds ---(overall)".format(time.time() - overall_start))  #opt





## timing
d = list(  zip( times.keys(), [np.mean(np.array(times[key])) for key in times], times.values() )  )
time_df, time_fname = pd.DataFrame(data=d, columns = ['Step', 'Avg Time', 'Times']), 'Step 1 Times {}'.format(nums_)
time_df.to_csv('{}.csv'.format(time_fname))
print('--- Times exported as {} ---'.format(time_fname)) # opt

This is the first file/a file with header descriptors
Not first file, using descriptors from book_dataset_0 as header
read 202 rows
NA rows removed
137 rows left
] removed in last column
[ removed in nAcid
2958 columns
--- 9.8654 seconds ---
numeric columns separated from the indices and smile columns
2956 columns left
inf replaced with col max value
cols with 2 unique values and below removed
1670 columns left
--- 1.3671 seconds ---
column mean calculated
--- 0.0038 seconds ---
all median within assumption, safe to reject mean/median == np.inf
1670 columns left
cols with mean/median > 300 removed; if median = 0, cols with mean > 300 removed
1368 columns left
--- 0.1808 seconds ---
1368 columns left
cols with |std/mean| < 0.1 removed
1318 columns left
rejoined with the indices and smile columns
File exported as Filtered 0_14
--- 0.5596 seconds ---
--- 11.9788 seconds ---(overall)
